# PyTorch: Using MLflow and Optuna for experiment tracking and hyperparameter optimization

## Imports

In [17]:
import os

import optuna
import mlflow
import torch

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
from mlflow import pytorch
from pprint import pformat
from urllib.parse import urlparse

## Network 

In [18]:
class Net(nn.Module):
    def __init__(self, dropout=0.0):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout2d(dropout)
        self.dropout2 = nn.Dropout2d(dropout)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output
    

## Train and validation functions

In [19]:
# Training loop
def train(options, model, device, train_loader, optimizer, epoch):
    model.train()
    train_set_size = len(train_loader.dataset)
    num_batches = len(train_loader)
    train_loss = 0.0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        train_loss += loss.item()
        loss.backward()
        optimizer.step()
        if batch_idx % options["log_interval"] == 0:
            batch_size = len(data)
            print(f"Train Epoch: {epoch} [{batch_idx * batch_size}/{train_set_size} "
                  f"({100. * batch_idx / num_batches:.0f}%)]\tLoss: {loss.item():.6f}")
    avg_train_loss = train_loss / num_batches
    return avg_train_loss

# Validation loop
def validate(model, device, test_loader):
    model.eval()
    test_set_size = len(test_loader.dataset)
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= test_set_size

    print(f"Test set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{test_set_size} "
          f"({100. * correct / test_set_size:.0f}%)\n")
    return test_loss

## Main

In [20]:
# Get the local path of the active mlflow run to save artifacts to
def get_artifact_path(active_run):
    parsed_uri = urlparse(active_run.info.artifact_uri)
    artifact_path = os.path.abspath(os.path.join(parsed_uri.netloc, parsed_uri.path))
    return artifact_path

In [21]:
# Obtain hyperparameters for this trial
def suggest_hyperparameters(trial):
    # Obtain the learning rate on a logarithmic scale
    lr = trial.suggest_float("lr", 1e-4, 1e-1, log=True)
    # Obtain the dropout ratio in a range from 0.0 to 0.9 with step size 0.1
    dropout = trial.suggest_float("dropout", 0.0, 0.9, step=0.1)
    # Obtain the batch size (as power of 2)
    batch_size = 2 ** trial.suggest_int("batch_size_power", 5, 8, step=1)
    # Obtain the optimizer to use by name
    optimizer_name = trial.suggest_categorical("optimizer_name", ["Adam", "Adadelta"])

    print(f"Suggested hyperparameters: \n{pformat(trial.params)}")
    # Log the obtained trial parameters using mlflow
    mlflow.log_params(trial.params)
    return lr, dropout, optimizer_name, batch_size

In [22]:
def get_mnist_dataloaders(batch_size=8):
    # Load the MNIST train and test datasets and save them to ./data
    mnist_train = datasets.MNIST('./data', train=True, download=True, transform=transforms.Compose([
                                               transforms.ToTensor(),
                                               transforms.Normalize((0.1307,), (0.3081,))
                                           ]))
    train_loader = torch.utils.data.DataLoader(mnist_train,
                                                       batch_size=batch_size,
                                                       shuffle=True)
    mnist_test = datasets.MNIST('./data', train=False, download=True, transform=transforms.Compose([
                                               transforms.ToTensor(),
                                               transforms.Normalize((0.1307,), (0.3081,))
                                           ]))
    val_loader = torch.utils.data.DataLoader(mnist_test,
                                                      batch_size=1000,
                                                      shuffle=True)
    return train_loader, val_loader

In [26]:
def objective(trial, experiment, options=None):
    # Initialize the best validation loss, which is the value to be minimized by the network
    best_val_loss = float('Inf')
    
    # Start mlflow run
    with mlflow.start_run(experiment_id=experiment.experiment_id):
        # Use mlflow to log experiment options
        mlflow.log_params(options)
        
        # Get hyperparameter suggestions created by optuna
        lr, dropout, optimizer_name, batch_size = suggest_hyperparameters(trial)
        
        print(f"\n**************************")

        active_run = mlflow.active_run()
        print(f"Starting run {active_run.info.run_id} and trial {trial.number}")

        # Parse the active mlflow run's artifact_uri and convert it into a system path
        parsed_uri = urlparse(active_run.info.artifact_uri)
        artifact_path = os.path.abspath(os.path.join(parsed_uri.netloc, parsed_uri.path))
        print(f"Artifact path for this run: {artifact_path}")
        
        # Use CUDA if GPU is available, else CPU
        use_cuda = options["use_cuda"] and torch.cuda.is_available()
        device = torch.device("cuda" if use_cuda else "cpu")
        # Log mlflow device parameter
        mlflow.log_param("device", device)

        # Obtain the MNIST train and validation loaders using a helper function
        train_loader, val_loader = get_mnist_dataloaders()
        
        # Initialize network
        model = Net(dropout=dropout).to(device)

        # Pick an optimizer based on optuna's parameter suggestion
        if optimizer_name == "Adam":
            optimizer = optim.Adam(model.parameters(), lr=lr)
        if optimizer_name == "Adadelta":
            optimizer = optim.Adadelta(model.parameters(), lr=lr)
        scheduler = StepLR(optimizer, step_size=1, gamma=0.7)

        # Network training & validation loop
        for epoch in range(0, options["epochs"]):
            avg_train_loss = train(options, model, device, train_loader, optimizer, epoch)
            avg_val_loss = validate(model, device, val_loader)
            
            if avg_val_loss <= best_val_loss:
                best_val_loss = avg_val_loss
            
            # Report intermediate objective value.
            trial.report(avg_val_loss, step=epoch)
            
            # Handle pruning based on the intermediate value.
            if trial.should_prune():
                raise optuna.TrialPruned()

            # Log average train and test set loss for the current epoch using mlflow
            mlflow.log_metric("avg_train_losses", avg_train_loss, step=epoch)
            mlflow.log_metric("avg_val_loss", avg_val_loss, step=epoch)
            scheduler.step()

        # Save the final network model to the current mlflow run's directory 
        if options["save_model"]:
            pytorch.save_model(model, f"{artifact_path}/mnist_model")

    # Return the best validation loss
    return best_val_loss

In [27]:
def main():
    # Experiment options
    options = {
        "experiment_name": "pytorch-optuna-mlflow",
        "epochs": 2,
        "use_cuda": False,
        "log_interval": 10,
        "save_model": True
    }

    # Create mlflow experiment if it doesn't exist already
    experiment_name = options["experiment_name"]
    experiment = mlflow.get_experiment_by_name(experiment_name)
    if experiment is None:
        mlflow.create_experiment(experiment_name)
        experiment = mlflow.get_experiment_by_name(experiment_name)
    mlflow.set_experiment(experiment_name)

    # Propagate logs to the root logger.
    optuna.logging.set_verbosity(verbosity=optuna.logging.INFO)

    # Create the optuna study which shares the experiment name
    study = optuna.create_study(study_name=experiment_name, direction="minimize")
    study.optimize(lambda trial: objective(trial, experiment, options), n_trials=5)

    # Filter optuna trials by state
    pruned_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.PRUNED]
    complete_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE]

    print("\n++++++++++++++++++++++++++++++++++\n")
    print("Study statistics: ")
    print("  Number of finished trials: ", len(study.trials))
    print("  Number of pruned trials: ", len(pruned_trials))
    print("  Number of complete trials: ", len(complete_trials))

    print("Best trial:")
    trial = study.best_trial

    print("  Trial number: ", trial.number)
    print("  Loss (trial value): ", trial.value)

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))


In [28]:
main()



[I 2020-11-16 13:27:39,449] A new study created in memory with name: pytorch-optuna-mlflow


Suggested hyperparameters: 
{'batch_size_power': 7,
 'dropout': 0.5,
 'lr': 0.00037477516268037106,
 'optimizer_name': 'Adam'}

**************************
Starting run 68725cd95b4a4c1ab1229e9517f49f8c and trial 0
Artifact path for this run: /Users/sstoppel/PycharmProjects/pytorch-mlflow-optuna/mlruns/1/68725cd95b4a4c1ab1229e9517f49f8c/artifacts
Train Epoch: 0 [0/60000 (0%)]	Loss: 2.365848
Train Epoch: 0 [80/60000 (0%)]	Loss: 2.234093
Train Epoch: 0 [160/60000 (0%)]	Loss: 1.997030
Train Epoch: 0 [240/60000 (0%)]	Loss: 1.915705
Train Epoch: 0 [320/60000 (1%)]	Loss: 1.735853
Train Epoch: 0 [400/60000 (1%)]	Loss: 1.532899
Train Epoch: 0 [480/60000 (1%)]	Loss: 1.433627
Train Epoch: 0 [560/60000 (1%)]	Loss: 1.364733
Train Epoch: 0 [640/60000 (1%)]	Loss: 1.041317
Train Epoch: 0 [720/60000 (1%)]	Loss: 1.344950
Train Epoch: 0 [800/60000 (1%)]	Loss: 0.561619
Train Epoch: 0 [880/60000 (1%)]	Loss: 1.169825
Train Epoch: 0 [960/60000 (2%)]	Loss: 0.593951
Train Epoch: 0 [1040/60000 (2%)]	Loss: 0.4180

Train Epoch: 0 [12960/60000 (22%)]	Loss: 1.083543
Train Epoch: 0 [13040/60000 (22%)]	Loss: 1.033233
Train Epoch: 0 [13120/60000 (22%)]	Loss: 1.170511
Train Epoch: 0 [13200/60000 (22%)]	Loss: 0.057174
Train Epoch: 0 [13280/60000 (22%)]	Loss: 0.034563
Train Epoch: 0 [13360/60000 (22%)]	Loss: 0.030227
Train Epoch: 0 [13440/60000 (22%)]	Loss: 0.151959
Train Epoch: 0 [13520/60000 (23%)]	Loss: 0.211424
Train Epoch: 0 [13600/60000 (23%)]	Loss: 1.039269
Train Epoch: 0 [13680/60000 (23%)]	Loss: 0.559794
Train Epoch: 0 [13760/60000 (23%)]	Loss: 0.147187
Train Epoch: 0 [13840/60000 (23%)]	Loss: 0.648556
Train Epoch: 0 [13920/60000 (23%)]	Loss: 0.612407
Train Epoch: 0 [14000/60000 (23%)]	Loss: 0.010766
Train Epoch: 0 [14080/60000 (23%)]	Loss: 0.113312
Train Epoch: 0 [14160/60000 (24%)]	Loss: 0.157550
Train Epoch: 0 [14240/60000 (24%)]	Loss: 0.074537
Train Epoch: 0 [14320/60000 (24%)]	Loss: 0.593373
Train Epoch: 0 [14400/60000 (24%)]	Loss: 0.506187
Train Epoch: 0 [14480/60000 (24%)]	Loss: 0.802885


Train Epoch: 0 [26080/60000 (43%)]	Loss: 0.035274
Train Epoch: 0 [26160/60000 (44%)]	Loss: 0.003355
Train Epoch: 0 [26240/60000 (44%)]	Loss: 0.139983
Train Epoch: 0 [26320/60000 (44%)]	Loss: 0.032358
Train Epoch: 0 [26400/60000 (44%)]	Loss: 0.065052
Train Epoch: 0 [26480/60000 (44%)]	Loss: 0.082687
Train Epoch: 0 [26560/60000 (44%)]	Loss: 1.001295
Train Epoch: 0 [26640/60000 (44%)]	Loss: 0.316128
Train Epoch: 0 [26720/60000 (45%)]	Loss: 0.049887
Train Epoch: 0 [26800/60000 (45%)]	Loss: 0.025671
Train Epoch: 0 [26880/60000 (45%)]	Loss: 0.414160
Train Epoch: 0 [26960/60000 (45%)]	Loss: 0.018881
Train Epoch: 0 [27040/60000 (45%)]	Loss: 0.218139
Train Epoch: 0 [27120/60000 (45%)]	Loss: 0.014449
Train Epoch: 0 [27200/60000 (45%)]	Loss: 0.346018
Train Epoch: 0 [27280/60000 (45%)]	Loss: 0.003575
Train Epoch: 0 [27360/60000 (46%)]	Loss: 0.270200
Train Epoch: 0 [27440/60000 (46%)]	Loss: 0.016207
Train Epoch: 0 [27520/60000 (46%)]	Loss: 0.024065
Train Epoch: 0 [27600/60000 (46%)]	Loss: 0.707592


Train Epoch: 0 [39200/60000 (65%)]	Loss: 0.174110
Train Epoch: 0 [39280/60000 (65%)]	Loss: 0.198922
Train Epoch: 0 [39360/60000 (66%)]	Loss: 0.014403
Train Epoch: 0 [39440/60000 (66%)]	Loss: 0.012076
Train Epoch: 0 [39520/60000 (66%)]	Loss: 0.019474
Train Epoch: 0 [39600/60000 (66%)]	Loss: 0.594651
Train Epoch: 0 [39680/60000 (66%)]	Loss: 0.002146
Train Epoch: 0 [39760/60000 (66%)]	Loss: 0.119020
Train Epoch: 0 [39840/60000 (66%)]	Loss: 0.011129
Train Epoch: 0 [39920/60000 (67%)]	Loss: 0.008799
Train Epoch: 0 [40000/60000 (67%)]	Loss: 0.026461
Train Epoch: 0 [40080/60000 (67%)]	Loss: 0.064456
Train Epoch: 0 [40160/60000 (67%)]	Loss: 0.152025
Train Epoch: 0 [40240/60000 (67%)]	Loss: 0.114539
Train Epoch: 0 [40320/60000 (67%)]	Loss: 0.304514
Train Epoch: 0 [40400/60000 (67%)]	Loss: 0.344423
Train Epoch: 0 [40480/60000 (67%)]	Loss: 0.796829
Train Epoch: 0 [40560/60000 (68%)]	Loss: 0.093069
Train Epoch: 0 [40640/60000 (68%)]	Loss: 0.160625
Train Epoch: 0 [40720/60000 (68%)]	Loss: 0.046291


...

Train Epoch: 1 [31840/60000 (53%)]	Loss: 2.047773
Train Epoch: 1 [31920/60000 (53%)]	Loss: 1.827752
Train Epoch: 1 [32000/60000 (53%)]	Loss: 2.045801
Train Epoch: 1 [32080/60000 (53%)]	Loss: 1.972765
Train Epoch: 1 [32160/60000 (54%)]	Loss: 2.170295
Train Epoch: 1 [32240/60000 (54%)]	Loss: 2.031839
Train Epoch: 1 [32320/60000 (54%)]	Loss: 1.879221
Train Epoch: 1 [32400/60000 (54%)]	Loss: 2.025820
Train Epoch: 1 [32480/60000 (54%)]	Loss: 1.967496
Train Epoch: 1 [32560/60000 (54%)]	Loss: 1.989304
Train Epoch: 1 [32640/60000 (54%)]	Loss: 2.123090
Train Epoch: 1 [32720/60000 (55%)]	Loss: 2.169753
Train Epoch: 1 [32800/60000 (55%)]	Loss: 2.034132
Train Epoch: 1 [32880/60000 (55%)]	Loss: 2.149611
Train Epoch: 1 [32960/60000 (55%)]	Loss: 2.007267
Train Epoch: 1 [33040/60000 (55%)]	Loss: 2.023766
Train Epoch: 1 [33120/60000 (55%)]	Loss: 2.147366
Train Epoch: 1 [33200/60000 (55%)]	Loss: 2.035936
Train Epoch: 1 [33280/60000 (55%)]	Loss: 2.110258
Train Epoch: 1 [33360/60000 (56%)]	Loss: 2.100674


Train Epoch: 1 [44960/60000 (75%)]	Loss: 2.093871
Train Epoch: 1 [45040/60000 (75%)]	Loss: 2.096897
Train Epoch: 1 [45120/60000 (75%)]	Loss: 1.963718
Train Epoch: 1 [45200/60000 (75%)]	Loss: 2.032878
Train Epoch: 1 [45280/60000 (75%)]	Loss: 1.693751
Train Epoch: 1 [45360/60000 (76%)]	Loss: 2.071849
Train Epoch: 1 [45440/60000 (76%)]	Loss: 2.052261
Train Epoch: 1 [45520/60000 (76%)]	Loss: 2.090402
Train Epoch: 1 [45600/60000 (76%)]	Loss: 2.053780
Train Epoch: 1 [45680/60000 (76%)]	Loss: 2.054334
Train Epoch: 1 [45760/60000 (76%)]	Loss: 1.992781
Train Epoch: 1 [45840/60000 (76%)]	Loss: 1.820039
Train Epoch: 1 [45920/60000 (77%)]	Loss: 1.806860
Train Epoch: 1 [46000/60000 (77%)]	Loss: 2.046597
Train Epoch: 1 [46080/60000 (77%)]	Loss: 1.970041
Train Epoch: 1 [46160/60000 (77%)]	Loss: 2.085802
Train Epoch: 1 [46240/60000 (77%)]	Loss: 2.086844
Train Epoch: 1 [46320/60000 (77%)]	Loss: 2.156828
Train Epoch: 1 [46400/60000 (77%)]	Loss: 1.816674
Train Epoch: 1 [46480/60000 (77%)]	Loss: 1.854023


Train Epoch: 1 [58080/60000 (97%)]	Loss: 2.005595
Train Epoch: 1 [58160/60000 (97%)]	Loss: 1.972908
Train Epoch: 1 [58240/60000 (97%)]	Loss: 1.914680
Train Epoch: 1 [58320/60000 (97%)]	Loss: 1.765197
Train Epoch: 1 [58400/60000 (97%)]	Loss: 1.794406
Train Epoch: 1 [58480/60000 (97%)]	Loss: 2.057373
Train Epoch: 1 [58560/60000 (98%)]	Loss: 2.030116
Train Epoch: 1 [58640/60000 (98%)]	Loss: 1.829967
Train Epoch: 1 [58720/60000 (98%)]	Loss: 1.899758
Train Epoch: 1 [58800/60000 (98%)]	Loss: 1.772795
Train Epoch: 1 [58880/60000 (98%)]	Loss: 1.978617
Train Epoch: 1 [58960/60000 (98%)]	Loss: 1.739035
Train Epoch: 1 [59040/60000 (98%)]	Loss: 2.019085
Train Epoch: 1 [59120/60000 (99%)]	Loss: 1.984200
Train Epoch: 1 [59200/60000 (99%)]	Loss: 1.871974
Train Epoch: 1 [59280/60000 (99%)]	Loss: 1.975207
Train Epoch: 1 [59360/60000 (99%)]	Loss: 1.876127
Train Epoch: 1 [59440/60000 (99%)]	Loss: 1.956029
Train Epoch: 1 [59520/60000 (99%)]	Loss: 2.070057
Train Epoch: 1 [59600/60000 (99%)]	Loss: 2.060611


[I 2020-11-16 14:38:49,003] Trial 4 finished with value: 1.8854495361328125 and parameters: {'lr': 0.00013298862799605266, 'dropout': 0.4, 'batch_size_power': 7, 'optimizer_name': 'Adadelta'}. Best is trial 0 with value: 0.04096794166564941.


Test set: Average loss: 1.8854, Accuracy: 6848/10000 (68%)


++++++++++++++++++++++++++++++++++

Study statistics: 
  Number of finished trials:  5
  Number of pruned trials:  0
  Number of complete trials:  5
Best trial:
  Trial number:  0
  Loss (trial value):  0.04096794166564941
  Params: 
    lr: 0.00037477516268037106
    dropout: 0.5
    batch_size_power: 7
    optimizer_name: Adam
